In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 181 kB/s             


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


model_1 = create_model_V1()
model_1.summary()

2022-04-10 19:02:33.673593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 19:02:33.756405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 19:02:33.757165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 19:02:33.758308: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3277056   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
model_test(5, 1, create_model_V1, (48,48), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


2022-04-10 19:02:47.526893: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-10 19:02:48.869597: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


718/718 - 143s - loss: 6.0778 - accuracy: 0.2030 - val_loss: 2.2949 - val_accuracy: 0.2281

Epoch 00001: val_accuracy improved from -inf to 0.22814, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_1.h5


225/225 [==============================] - 34s 150ms/step - loss: 2.2870 - accuracy: 0.2295
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 21s - loss: 5.7556 - accuracy: 0.2077 - val_loss: 2.7219 - val_accuracy: 0.1832

Epoch 00001: val_accuracy improved from -inf to 0.18321, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_2.h5


225/225 [==============================] - 5s 22ms/step - loss: 2.7340 - accuracy: 0.1864
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 21s - loss: 5.4851 - accuracy: 0.2030 - val_loss: 2.5988 - val_accuracy: 0.1867

Epoch 00001: val_accuracy improved from -inf to 0.18669, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_3.h5


225/225 [==============================] - 5s 23ms/step - loss: 2.6057 - accuracy: 0.1904
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 21s - loss: 6.5801 - accuracy: 0.1979 - val_loss: 2.4080 - val_accuracy: 0.2060

Epoch 00001: val_accuracy improved from -inf to 0.20603, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_4.h5


225/225 [==============================] - 5s 23ms/step - loss: 2.4046 - accuracy: 0.2108
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
718/718 - 20s - loss: 5.6937 - accuracy: 0.1998 - val_loss: 2.1805 - val_accuracy: 0.2174

Epoch 00001: val_accuracy improved from -inf to 0.21738, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_5.h5


225/225 [==============================] - 5s 24ms/step - loss: 2.2039 - accuracy: 0.2301


In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 160
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

94781440/94765736 [==============================] - 32s 0us/step
175


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343 

In [17]:
model_test(5, 50, transfer_learning_model_resnet_50, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 28s - loss: 3.3719 - accuracy: 0.2435 - val_loss: 2.2020 - val_accuracy: 0.2658

Epoch 00001: val_accuracy improved from -inf to 0.26576, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



Epoch 2/50
718/718 - 23s - loss: 2.4658 - accuracy: 0.3231 - val_loss: 2.1357 - val_accuracy: 0.2997

Epoch 00002: val_accuracy improved from 0.26576 to 0.29972, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 3/50
718/718 - 24s - loss: 2.0790 - accuracy: 0.3947 - val_loss: 1.9440 - val_accuracy: 0.3551

Epoch 00003: val_accuracy improved from 0.29972 to 0.35510, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 4/50
718/718 - 24s - loss: 1.7207 - accuracy: 0.4725 - val_loss: 1.6215 - val_accuracy: 0.4673

Epoch 00004: val_accuracy improved from 0.35510 to 0.46726, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 5/50
718/718 - 26s - loss: 1.3996 - accuracy: 0.5570 - val_loss: 1.8310 - val_accuracy: 0.4192

E

225/225 [==============================] - 6s 27ms/step - loss: 3.2165 - accuracy: 0.5148
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 27s - loss: 3.4572 - accuracy: 0.2451 - val_loss: 2.1020 - val_accuracy: 0.3037

Epoch 00001: val_accuracy improved from -inf to 0.30373, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 2/50
718/718 - 23s - loss: 2.4852 - accuracy: 0.3232 - val_loss: 1.9673 - val_accuracy: 0.3366

Epoch 00002: val_accuracy improved from 0.30373 to 0.33664, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 3/50
718/718 - 24s - loss: 2.0522 - accuracy: 0.3988 - val_loss: 1.8499 - val_accuracy: 0.3790

Epoch 00003: val_accuracy improved from 0.33664 to 0.37896, saving model to /kaggle/wo

225/225 [==============================] - 6s 27ms/step - loss: 3.1673 - accuracy: 0.5077
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 27s - loss: 3.4200 - accuracy: 0.2522 - val_loss: 2.2763 - val_accuracy: 0.2710

Epoch 00001: val_accuracy improved from -inf to 0.27099, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 2/50
718/718 - 23s - loss: 2.4641 - accuracy: 0.3270 - val_loss: 2.1019 - val_accuracy: 0.3065

Epoch 00002: val_accuracy improved from 0.27099 to 0.30651, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 3/50
718/718 - 23s - loss: 2.0306 - accuracy: 0.4044 - val_loss: 1.8378 - val_accuracy: 0.3654

Epoch 00003: val_accuracy improved from 0.30651 to 0.36538, saving model to /kaggle/wo

225/225 [==============================] - 6s 27ms/step - loss: 3.2762 - accuracy: 0.5226
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 28s - loss: 3.2859 - accuracy: 0.2490 - val_loss: 2.2695 - val_accuracy: 0.2687

Epoch 00001: val_accuracy improved from -inf to 0.26872, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 2/50
718/718 - 24s - loss: 2.4327 - accuracy: 0.3250 - val_loss: 1.9136 - val_accuracy: 0.3276

Epoch 00002: val_accuracy improved from 0.26872 to 0.32759, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 3/50
718/718 - 24s - loss: 2.0195 - accuracy: 0.4022 - val_loss: 1.7686 - val_accuracy: 0.4028

Epoch 00003: val_accuracy improved from 0.32759 to 0.40282, saving model to /kaggle/wo

225/225 [==============================] - 6s 28ms/step - loss: 2.4555 - accuracy: 0.5065
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 28s - loss: 3.3051 - accuracy: 0.2429 - val_loss: 2.2125 - val_accuracy: 0.2515

Epoch 00001: val_accuracy improved from -inf to 0.25152, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 2/50
718/718 - 24s - loss: 2.4307 - accuracy: 0.3238 - val_loss: 1.9222 - val_accuracy: 0.3412

Epoch 00002: val_accuracy improved from 0.25152 to 0.34123, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 3/50
718/718 - 25s - loss: 2.0494 - accuracy: 0.3958 - val_loss: 1.8000 - val_accuracy: 0.3783

Epoch 00003: val_accuracy improved from 0.34123 to 0.37833, saving model to /kaggle/wo

225/225 [==============================] - 7s 30ms/step - loss: 3.3830 - accuracy: 0.5104


In [18]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 16
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step
19


In [19]:
transfer_model_vgg_16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 3591  

In [20]:
model_test(5, 50, transfer_learning_model_vgg_16, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 23s - loss: 3.7815 - accuracy: 0.1682 - val_loss: 1.8681 - val_accuracy: 0.1684

Epoch 00001: val_accuracy improved from -inf to 0.16841, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 2/50
718/718 - 22s - loss: 3.2147 - accuracy: 0.1680 - val_loss: 1.8912 - val_accuracy: 0.1682

Epoch 00002: val_accuracy did not improve from 0.16841
Epoch 3/50
718/718 - 21s - loss: 2.9529 - accuracy: 0.1681 - val_loss: 2.1907 - val_accuracy: 0.1682

Epoch 00003: val_accuracy did not improve from 0.16841
Epoch 4/50
718/718 - 21s - loss: 2.8580 - accuracy: 0.1682 - val_loss: 2.1036 - val_accuracy: 0.1682

Epoch 00004: val_accuracy did not improve from 0.16841
Epoch 5/50
718/718 - 22s - loss: 2.7477 - accuracy: 0.1685 - val_loss: 2.0931 - val_accuracy: 0.1681

Epoch 00005

225/225 [==============================] - 7s 31ms/step - loss: 2.0499 - accuracy: 0.5210
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 22s - loss: 4.0531 - accuracy: 0.1673 - val_loss: 2.2008 - val_accuracy: 0.1681

Epoch 00001: val_accuracy improved from -inf to 0.16806, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 2/50
718/718 - 22s - loss: 3.0712 - accuracy: 0.1688 - val_loss: 2.3155 - val_accuracy: 0.1682

Epoch 00002: val_accuracy improved from 0.16806 to 0.16823, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 3/50
718/718 - 22s - loss: 2.8883 - accuracy: 0.1686 - val_loss: 2.0962 - val_accuracy: 0.1682

Epoch 00003: val_accuracy did not improve from 0.16823
Epoch 4/50
718/718 - 22s - loss: 2.7959 - ac

225/225 [==============================] - 6s 26ms/step - loss: 2.5203 - accuracy: 0.5422
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 26s - loss: 3.8154 - accuracy: 0.1684 - val_loss: 2.2649 - val_accuracy: 0.1686

Epoch 00001: val_accuracy improved from -inf to 0.16858, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 2/50
718/718 - 25s - loss: 2.9916 - accuracy: 0.1682 - val_loss: 2.0131 - val_accuracy: 0.1684

Epoch 00002: val_accuracy did not improve from 0.16858
Epoch 3/50
718/718 - 24s - loss: 2.9026 - accuracy: 0.1685 - val_loss: 2.1966 - val_accuracy: 0.1686

Epoch 00003: val_accuracy did not improve from 0.16858
Epoch 4/50
718/718 - 22s - loss: 2.7570 - accuracy: 0.1685 - val_loss: 2.0078 - val_accuracy: 0.1688

Epoch 00004: val_accuracy improved from 0.16858 to 0.16876, saving model to /k

225/225 [==============================] - 7s 32ms/step - loss: 2.4213 - accuracy: 0.5372
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 23s - loss: 3.9920 - accuracy: 0.1675 - val_loss: 2.4323 - val_accuracy: 0.1686

Epoch 00001: val_accuracy improved from -inf to 0.16858, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 2/50
718/718 - 22s - loss: 2.9761 - accuracy: 0.1684 - val_loss: 2.1599 - val_accuracy: 0.1682

Epoch 00002: val_accuracy did not improve from 0.16858
Epoch 3/50
718/718 - 21s - loss: 2.7335 - accuracy: 0.1682 - val_loss: 2.1395 - val_accuracy: 0.1682

Epoch 00003: val_accuracy did not improve from 0.16858
Epoch 4/50
718/718 - 22s - loss: 2.6476 - accuracy: 0.1685 - val_loss: 2.0827 - val_accuracy: 0.1682

Epoch 00004: val_accuracy did not improve from 0.16858
Epoch 5/50
718/718 - 22

225/225 [==============================] - 6s 28ms/step - loss: 2.4242 - accuracy: 0.5350
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 22s - loss: 4.1553 - accuracy: 0.1669 - val_loss: 2.0349 - val_accuracy: 0.1684

Epoch 00001: val_accuracy improved from -inf to 0.16844, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 2/50
718/718 - 21s - loss: 3.2533 - accuracy: 0.1681 - val_loss: 2.0471 - val_accuracy: 0.1681

Epoch 00002: val_accuracy did not improve from 0.16844
Epoch 3/50
718/718 - 21s - loss: 2.9461 - accuracy: 0.1683 - val_loss: 1.9105 - val_accuracy: 0.1683

Epoch 00003: val_accuracy did not improve from 0.16844
Epoch 4/50
718/718 - 21s - loss: 2.8462 - accuracy: 0.1685 - val_loss: 2.1604 - val_accuracy: 0.1684

Epoch 00004: val_accuracy did not improve from 0.16844
Epoch 5/50
718/718 - 23

225/225 [==============================] - 6s 25ms/step - loss: 1.9512 - accuracy: 0.5469
